In [40]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import geopandas as gpd
pd.set_option('display.max_columns', 30)

In [2]:
pwd!

'/Users/suneethi/Documents/nssdatasc/dq4_nashville_metro/dq4-metro-beetles'

In [3]:
metro_trash=pd.read_excel('data/Trash Hauler Report - Nov 2017 - Nov 2019 Final.xlsx', skiprows=1)
metro=pd.read_csv('data/hubNashville__311__Service_Requests.csv')

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
metro=metro.drop(columns=['Request Origin', 'System of Origin ID', 'Contact Type', 
                              'Preferred Language', 'State Issue'])

In [5]:
metro.head()

,Request #,Status,Request Type,Subrequest Type,Additional Subrequest Type,Date / Time Opened,Date / Time Closed,Parent Request,Closed When Created,Address,City,Council District,ZIP,Latitude,Longitude,Mapped Location
0,108110,Closed,"Trash, Recycling & Litter",Cart Service,Trash - Request an Additional Cart,10/10/18 16:14,3/5/19 21:34,NaN,False,413 Freda Villa,MADISON,8.0,37115,36.243960,-86.727210,"(36.24396, -86.72721)"
1,122078,Closed,"Trash, Recycling & Litter",File a Trash or Recycling Complaint,Trash in Recycling Cart,12/7/18 14:50,12/7/18 14:50,NaN,True,"4906 Dakota Ave, Nashville, TN 37209, United S...",NASHVILLE,24.0,37209,36.147680,-86.846440,"(36.14768, -86.84644)"
2,113730,Closed,"Streets, Roads & Sidewalks",Request New Sign,NaN,11/1/18 10:00,11/6/18 11:15,NaN,True,"873 Granada Ave, Nashville, TN 37206, United S...",NASHVILLE,5.0,37206,36.189230,-86.756430,"(36.18923, -86.75643)"
3,124624,Closed,"Trash, Recycling & Litter",File a Trash or Recycling Complaint,Trash in Recycling Cart,12/19/18 17:47,12/19/18 17:47,NaN,False,4304 Lindawood Dr,NASHVILLE,34.0,37215,36.097154,-86.831878,"(36.0971543, -86.8318783)"
4,108241,Closed,"Streets, Roads & Sidewalks",Street Lighting,Report Outage,10/10/18 14:49,10/24/18 16:27,NaN,True,"336 Blackman Rd, Nashville, TN 37211, United S...",NASHVILLE,26.0,37211,36.074580,-86.746960,"(36.07458, -86.74696)"


In [6]:
metro=metro.rename(columns={'Request #': 'Request_id', 'Request Type':'Request_type',
                                'Subrequest Type': 'Subrequest_type', 
                                'Additional Subrequest Type': 'Additional_subrequest',
                               'Date / Time Opened' :'dt_opened', 'Date / Time Closed':'dt_closed',
                               'Council District': 'Council_dist', 'ZIP':'Zip', 'Latitude':'Lat', 
                               'Longitude':'Long', 'Mapped Location': 'Location'})

In [7]:
metro_misd=metro[(metro['Subrequest_type']=='Missed Pickup - Recycle Cart Service') | (metro['Subrequest_type']==
                                                                                'Missed Pickup - Trash Cart Service')
         | (metro['Subrequest_type']=='Missed Pickup - Cart Service') | (metro['Subrequest_type']=='Missed Pickup - Dumpster Service')]

In [8]:
metro_trash.head()

,Request Number,Date Opened,Request,Description,Incident Address,Zip Code,Trash Hauler,Trash Route,Council District,State Plan X,State Plan Y
0,25270,2017-11-01 07:09:00,Trash - Backdoor,"house with the wheel chair ramp, they share dr...",3817 Crouch Dr,37207.0,RED RIVER,3205,2.0,1.727970e+06,686779.478089
1,25274,2017-11-01 07:42:00,Trash - Curbside/Alley Missed Pickup,Curb/Trash miss Tuesday.,4028 Clarksville Pike,37218.0,RED RIVER,4202,1.0,1.721259e+06,685444.799565
2,25276,2017-11-01 07:53:00,Trash - Curbside/Alley Missed Pickup,Curb/trash miss Tuesday.,6528 Thunderbird Dr,37209.0,RED RIVER,4205,20.0,1.707027e+06,659887.471571
3,25307,2017-11-01 08:28:00,Trash - Curbside/Alley Missed Pickup,missed,2603 old matthews rd,37207.0,WASTE IND,2206,2.0,1.735692e+06,685027.245923
4,25312,2017-11-01 08:55:00,Trash - Curbside/Alley Missed Pickup,Missed the even side of the road.,604 croley dr,37209.0,RED RIVER,4203,20.0,1.710186e+06,664205.101066


In [9]:
metro_trash.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20226 entries, 0 to 20225
Data columns (total 11 columns):
Request Number      20226 non-null int64
Date Opened         20226 non-null datetime64[ns]
Request             20226 non-null object
Description         20197 non-null object
Incident Address    20217 non-null object
Zip Code            20151 non-null float64
Trash Hauler        19325 non-null object
Trash Route         19279 non-null object
Council District    20177 non-null float64
State Plan X        20198 non-null float64
State Plan Y        20198 non-null float64
dtypes: datetime64[ns](1), float64(4), int64(1), object(5)
memory usage: 1.7+ MB


In [10]:
metro_trash=metro_trash.rename(columns={'Request Number': 'Request_id', 'Date Opened' :'dt_opened', 
                                            'Incident Address': 'Address', 'Zip Code': 'Zip', 'Trash Hauler': 'Hauler'
                                           ,'Council District': 'Council_dist' })

In [11]:
metro_trash.shape

(20226, 11)

In [12]:
metro_trash.head()

,Request_id,dt_opened,Request,Description,Address,Zip,Hauler,Trash Route,Council_dist,State Plan X,State Plan Y
0,25270,2017-11-01 07:09:00,Trash - Backdoor,"house with the wheel chair ramp, they share dr...",3817 Crouch Dr,37207.0,RED RIVER,3205,2.0,1.727970e+06,686779.478089
1,25274,2017-11-01 07:42:00,Trash - Curbside/Alley Missed Pickup,Curb/Trash miss Tuesday.,4028 Clarksville Pike,37218.0,RED RIVER,4202,1.0,1.721259e+06,685444.799565
2,25276,2017-11-01 07:53:00,Trash - Curbside/Alley Missed Pickup,Curb/trash miss Tuesday.,6528 Thunderbird Dr,37209.0,RED RIVER,4205,20.0,1.707027e+06,659887.471571
3,25307,2017-11-01 08:28:00,Trash - Curbside/Alley Missed Pickup,missed,2603 old matthews rd,37207.0,WASTE IND,2206,2.0,1.735692e+06,685027.245923
4,25312,2017-11-01 08:55:00,Trash - Curbside/Alley Missed Pickup,Missed the even side of the road.,604 croley dr,37209.0,RED RIVER,4203,20.0,1.710186e+06,664205.101066


In [20]:
metro_trash=pd.merge(metro_trash, metro, left_on='Request_id', right_on='Request_id', how='inner')

In [21]:
metro_trash.head()

,Request_id,dt_opened_x,Request,Description,Address_x,Zip_x,Hauler,Trash Route,Council_dist_x,State Plan X,State Plan Y,Status_x,Request_type_x,Subrequest_type_x,dt_closed_x,...,Status_y,Request_type_y,Subrequest_type_y,Additional_subrequest,dt_opened_y,dt_closed_y,Parent Request_y,Closed When Created_y,Address,City_y,Council_dist,Zip,Lat_y,Long_y,Location_y
0,25270,2017-11-01 07:09:00,Trash - Backdoor,"house with the wheel chair ramp, they share dr...",3817 Crouch Dr,37207.0,RED RIVER,3205,2.0,1.727970e+06,686779.478089,Closed,"Trash, Recycling & Litter",Missed Pickup - Trash Cart Service,11/2/17 16:02,...,Closed,"Trash, Recycling & Litter",Missed Pickup - Trash Cart Service,Trash - Backdoor,11/1/17 7:09,11/2/17 16:02,NaN,True,3817 Crouch Dr,NASHVILLE,2.0,37207,36.21730,-86.81540,"(36.2173, -86.8154)"
1,25274,2017-11-01 07:42:00,Trash - Curbside/Alley Missed Pickup,Curb/Trash miss Tuesday.,4028 Clarksville Pike,37218.0,RED RIVER,4202,1.0,1.721259e+06,685444.799565,Closed,"Trash, Recycling & Litter",Missed Pickup - Trash Cart Service,11/2/17 16:02,...,Closed,"Trash, Recycling & Litter",Missed Pickup - Trash Cart Service,Trash - Curbside/Alley Missed Pickup,11/1/17 7:42,11/2/17 16:02,NaN,True,4028 Clarksville Pike,NASHVILLE,1.0,37218,36.21348,-86.83811,"(36.21348, -86.83811)"
2,25276,2017-11-01 07:53:00,Trash - Curbside/Alley Missed Pickup,Curb/trash miss Tuesday.,6528 Thunderbird Dr,37209.0,RED RIVER,4205,20.0,1.707027e+06,659887.471571,Closed,"Trash, Recycling & Litter",Missed Pickup - Trash Cart Service,11/2/17 16:00,...,Closed,"Trash, Recycling & Litter",Missed Pickup - Trash Cart Service,Trash - Curbside/Alley Missed Pickup,11/1/17 7:53,11/2/17 16:00,NaN,True,6528 Thunderbird Dr,NASHVILLE,20.0,37209,36.14293,-86.88557,"(36.14293, -86.88557)"
3,25307,2017-11-01 08:28:00,Trash - Curbside/Alley Missed Pickup,missed,2603 old matthews rd,37207.0,WASTE IND,2206,2.0,1.735692e+06,685027.245923,Closed,"Trash, Recycling & Litter",Missed Pickup - Trash Cart Service,11/7/17 4:03,...,Closed,"Trash, Recycling & Litter",Missed Pickup - Trash Cart Service,Trash - Curbside/Alley Missed Pickup,11/1/17 8:28,11/7/17 4:03,NaN,True,2603 old matthews rd,NASHVILLE,2.0,37207,36.21266,-86.78918,"(36.21266, -86.78918)"
4,25312,2017-11-01 08:55:00,Trash - Curbside/Alley Missed Pickup,Missed the even side of the road.,604 croley dr,37209.0,RED RIVER,4203,20.0,1.710186e+06,664205.101066,Closed,"Trash, Recycling & Litter",Missed Pickup - Trash Cart Service,11/2/17 16:00,...,Closed,"Trash, Recycling & Litter",Missed Pickup - Trash Cart Service,Trash - Curbside/Alley Missed Pickup,11/1/17 8:55,11/2/17 16:00,NaN,True,604 croley dr,NASHVILLE,20.0,37209,36.15487,-86.87500,"(36.15487, -86.875)"


In [15]:
metro_trash.shape

(20226, 26)

In [23]:
metro_trash=metro_trash.drop(columns=['Additional_subrequest', 'Council_dist_x', 'Zip_x', 'dt_opened_y', 'Address'])

In [28]:
metro_trash=metro_trash.rename(columns={'dt_opened_x': 'dt_opened', 'Request ': 'Request', 'Address_x': 'Address'})

In [26]:
metro_trash.head()

,Request_id,dt_opened,Request,Description,Address,Hauler,Trash Route,State Plan X,State Plan Y,Status_x,Request_type_x,Subrequest_type_x,dt_closed_x,Parent Request_x,Closed When Created_x,...,Lat_x,Long_x,Location_x,Status_y,Request_type_y,Subrequest_type_y,dt_closed_y,Parent Request_y,Closed When Created_y,City_y,Council_dist,Zip,Lat_y,Long_y,Location_y
0,25270,2017-11-01 07:09:00,Trash - Backdoor,"house with the wheel chair ramp, they share dr...",3817 Crouch Dr,RED RIVER,3205,1.727970e+06,686779.478089,Closed,"Trash, Recycling & Litter",Missed Pickup - Trash Cart Service,11/2/17 16:02,NaN,True,...,36.21730,-86.81540,"(36.2173, -86.8154)",Closed,"Trash, Recycling & Litter",Missed Pickup - Trash Cart Service,11/2/17 16:02,NaN,True,NASHVILLE,2.0,37207,36.21730,-86.81540,"(36.2173, -86.8154)"
1,25274,2017-11-01 07:42:00,Trash - Curbside/Alley Missed Pickup,Curb/Trash miss Tuesday.,4028 Clarksville Pike,RED RIVER,4202,1.721259e+06,685444.799565,Closed,"Trash, Recycling & Litter",Missed Pickup - Trash Cart Service,11/2/17 16:02,NaN,True,...,36.21348,-86.83811,"(36.21348, -86.83811)",Closed,"Trash, Recycling & Litter",Missed Pickup - Trash Cart Service,11/2/17 16:02,NaN,True,NASHVILLE,1.0,37218,36.21348,-86.83811,"(36.21348, -86.83811)"
2,25276,2017-11-01 07:53:00,Trash - Curbside/Alley Missed Pickup,Curb/trash miss Tuesday.,6528 Thunderbird Dr,RED RIVER,4205,1.707027e+06,659887.471571,Closed,"Trash, Recycling & Litter",Missed Pickup - Trash Cart Service,11/2/17 16:00,NaN,True,...,36.14293,-86.88557,"(36.14293, -86.88557)",Closed,"Trash, Recycling & Litter",Missed Pickup - Trash Cart Service,11/2/17 16:00,NaN,True,NASHVILLE,20.0,37209,36.14293,-86.88557,"(36.14293, -86.88557)"
3,25307,2017-11-01 08:28:00,Trash - Curbside/Alley Missed Pickup,missed,2603 old matthews rd,WASTE IND,2206,1.735692e+06,685027.245923,Closed,"Trash, Recycling & Litter",Missed Pickup - Trash Cart Service,11/7/17 4:03,NaN,True,...,36.21266,-86.78918,"(36.21266, -86.78918)",Closed,"Trash, Recycling & Litter",Missed Pickup - Trash Cart Service,11/7/17 4:03,NaN,True,NASHVILLE,2.0,37207,36.21266,-86.78918,"(36.21266, -86.78918)"
4,25312,2017-11-01 08:55:00,Trash - Curbside/Alley Missed Pickup,Missed the even side of the road.,604 croley dr,RED RIVER,4203,1.710186e+06,664205.101066,Closed,"Trash, Recycling & Litter",Missed Pickup - Trash Cart Service,11/2/17 16:00,NaN,True,...,36.15487,-86.87500,"(36.15487, -86.875)",Closed,"Trash, Recycling & Litter",Missed Pickup - Trash Cart Service,11/2/17 16:00,NaN,True,NASHVILLE,20.0,37209,36.15487,-86.87500,"(36.15487, -86.875)"


In [33]:
metro_trash.Request.value_counts

Trash - Curbside/Alley Missed Pickup    15028
Trash - Backdoor                         2629
Trash Collection Complaint               2312
Damage to Property                        257
Name: Request, dtype: int64

In [46]:
metro_trash.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20226 entries, 0 to 20225
Data columns (total 31 columns):
Request_id               20226 non-null int64
dt_opened                20226 non-null datetime64[ns]
Request                  20226 non-null object
Description              20197 non-null object
Address                  20217 non-null object
Hauler                   19325 non-null object
Trash Route              19279 non-null object
State Plan X             20198 non-null float64
State Plan Y             20198 non-null float64
Status_x                 20226 non-null object
Request_type_x           20226 non-null object
Subrequest_type_x        20226 non-null object
dt_closed_x              20131 non-null object
Parent Request_x         116 non-null float64
Closed When Created_x    20226 non-null bool
City_x                   20178 non-null object
Lat_x                    20201 non-null float64
Long_x                   20201 non-null float64
Location_x               20201 non-nu

In [43]:
sub='miss'

SyntaxError: invalid syntax (<ipython-input-43-e0e6da608ab0>, line 1)

In [44]:
metro_trash.apply(lambda row: row.astype(str).str.contains('miss').any(), axis=1)

0        False
1         True
2         True
3         True
4        False
         ...  
20221    False
20222    False
20223     True
20224     True
20225     True
Length: 20226, dtype: bool

In [45]:
keywords = ['miss', 'missed']

keywords_not_required = ['Damage to Property']

def check_keywords(df, keywords):
    contains_keywords = pd.DataFrame()
    for keyword in keywords:
        contains_keyword = pd.DataFrame()
        for column in df:
            if df[column].dtype not in ['float64', 'int64']:
                contains_keyword[column] = df[column].str.lower().str.contains(keyword.lower())
        contains_keywords[keyword] = contains_keyword.any(axis=1)
    return contains_keywords.astype('int64')

keyword_scores = check_keywords(metro_trash, keywords)
keyword_required_score = keyword_scores[keywords_required].sum(axis=1)




AttributeError: Can only use .str accessor with string values!